In [1]:
import copy
import random
import time
frontboard=[[1, 1, 1, 1, 1],[1, 0, 0, 0, 1],[1, 0, 0, 0, -1],[-1, 0, 0, 0, -1],[-1, -1, -1, -1, -1]]
def move(board,player): # Ham move cua de bai
    valueBoard=Value(board)
    global frontboard
    check = checkboard(board)
    if check!=None: return check
    else:
        bestchoice=-1
        arr=[]
        arrboard=[]
        if valueBoard==0 or valueBoard==16:return None
        mn=-17
        for i in range(5):
            for j in range(5):
                if board[i][j]==player: # Xet nhung diem co gia tri bang player, xem co the di duoc toi nhung cho nao
                    if (i+j)%2==0: # di chuyen cheo duoc
                        for m in range(i-1,i+2):
                            for n in range(j-1,j+2):
                                bcopy=copy.deepcopy(board)
                                child = calmove((i,j),(m,n),bcopy)
                                if child == None:continue
                                tmp = AlphaBeta(child,1,-17,17,False,player)
                                if mn<tmp: # Tim gia tri con lon nhat roi gan source voi des vao
                                    mn=tmp
                                    arr=[[(i,j),(m,n)]]
                                    arrboard=[child]
                                elif mn==tmp:
                                    arr.append([(i,j),(m,n)])
                                    arrboard.append(child)
                    else: # khong di chuyen cheo duoc
                        for des in [(i-1,j),(i,j-1),(i,j+1),(i+1,j)]: # khong di chuyen cheo duoc nen chi co 4 diem 
                            bcopy=copy.deepcopy(board)
                            child = calmove((i,j),des,bcopy)
                            if child == None:continue
                            tmp = AlphaBeta(child,1,-17,17,False,player)
                            if mn<tmp:
                                mn=tmp
                                arr=[[(i,j),des]]
                                arrboard=[child]
                            elif mn==tmp:
                                arr.append([(i,j),des])
                                arrboard.append(child)
        k=random.randint(0,len(arr)-1)
        frontboard = arrboard[k]
        return (arr[k][0],arr[k][1])
def checkboard(board):
    global frontboard
    if Value(board)==Value(frontboard):
        for i in range(5):
            for j in range(5):
                if frontboard[i][j]==-1 and board[i][j]==0:
                    for (m,n) in AdjList(i,j,board):
                        if board[m][n]==1:
                            bcopy=copy.deepcopy(board)
                            newboard=calmove((m,n),(i,j),bcopy)
                            if newboard!=None:
                                if Value(newboard)>Value(board):
                                    frontboard = newboard
                                    return ((m,n),(i,j))
    return None
def minmove(board,player):
    if Value(board)==0 or Value(board)==16:return 0
    mn=17
    for i in range(5):
        for j in range(5):
            if board[i][j]==-player: # Xet nhung diem co gia tri bang player, xem co the di duoc toi nhung cho nao
                if (i+j)%2==0: # di chuyen cheo duoc
                    for m in range(i-1,i+2):
                        for n in range(j-1,j+2):
                            bcopy=copy.deepcopy(board)
                            child = calmove((i,j),(m,n),bcopy)
                            if child == None:continue
                            tmp = AlphaBeta(child,1,-17,17,False,player)
                            if mn>tmp: # Tim gia tri con lon nhat roi gan source voi des vao
                                mn=tmp
                else: # khong di chuyen cheo duoc
                    for des in [(i-1,j),(i,j-1),(i,j+1),(i+1,j)]: # khong di chuyen cheo duoc nen chi co 4 diem 
                        bcopy=copy.deepcopy(board)
                        child = calmove((i,j),des,bcopy)
                        if child == None:continue
                        tmp = AlphaBeta(child,1,-17,17,False,player)
                        if mn<tmp:
                            mn=tmp
    return mn
def checkOnBoard(x,y): #kiem tra xem co nam trong bang hay khong 
    return 0<=x<5 and 0<=y<5
def calmove(source,des,s): # run simulator de xem gia tri nao toi uu
    sx,sy,dx,dy = source[0],source[1],des[0],des[1]
    if checkOnBoard(dx,dy):
        if s[dx][dy]==0:
            s[dx][dy],s[sx][sy]=s[sx][sy],0
            update(s,(dx,dy)) # move roi thi update
            return s
    return None
def AdjList1(x,y,s): # tim cac diem ke can cua (x,y)
    adj_li=[]
    if (x+y)%2==0:
        for i in range(x-1,x+2):
            for j in range(y-1,y+2):
                if checkOnBoard(i,j) and s[i][j]==-1:adj_li.append((i,j))
    else:
        for des in [(x-1,y),(x,y-1),(x,y+1),(x+1,y)]:
            i,j=des[0],des[1]
            if checkOnBoard(i,j) and s[i][j]==-1:adj_li.append((i,j))
    return adj_li
def AdjList(x,y,s): # tim cac diem ke can cua (x,y)
    adj_li=[]
    if (x+y)%2==0:
        for i in range(x-1,x+2):
            for j in range(y-1,y+2):
                if checkOnBoard(i,j) and s[i][j]!=-s[x][y]:adj_li.append((i,j))
    else:
        for des in [(x-1,y),(x,y-1),(x,y+1),(x+1,y)]:
            i,j=des[0],des[1]
            if checkOnBoard(i,j) and s[i][j]!=-s[x][y]:adj_li.append((i,j))
    return adj_li
def update(s,des): 
    i,j=des[0],des[1]
    k=i-1
    for m in range(j-1,j+2): # xet truong hop Ganh
        if checkOnBoard(k,m) and checkOnBoard(i*2-k,j*2-m):
            if s[k][m]==s[i*2-k][j*2-m]==-s[i][j]:
                s[k][m]=s[i][j]
                s[i*2-k][j*2-m]=s[i][j]
    if checkOnBoard(i,j-1) and checkOnBoard(i,j+1):
        if s[i][j-1]==s[i][j+1]==-s[i][j]:
            s[i][j-1]=s[i][j]
            s[i][j+1]=s[i][j]
    for i in range(5): # xet truong hop vay
        for j in range(5):
            if s[i][j]!=0:
                adjList=AdjList(i,j,s)
                for des in adjList:
                    if s[des[0]][des[1]]==0:break # Neu co ke can voi 0 thi khong bi doi gia tri
                    else:
                        for adj in AdjList(des[0],des[1],s):
                            if adj not in adjList:adjList.append(adj) # chay vong lap de xet het cac dinh ke can cua cac dinh ke can
                else:s[i][j]=-s[i][j] # neu khong tim ra 0 thi bi doi gia tri
            
def Value(s): # tra ve so luong "1" co tren board
    count=0
    for i in range(5):
        for j in range(5):
            if s[i][j]==1:count+=1
    return count
def AlphaBeta(s,depth,a,b,mp,player):# mp: maximizingPlayer; Giai thuat AlphaBeta
    d=depth
    if Value(s)==0 or Value(s)==16 or d==0:return Value(s)
    if mp==True:
        for i in range(5):
            for j in range(5):
                if s[i][j]==player:
                    if (i+j)%2==0: # di chuyen cheo duoc
                        for m in range(i-1,i+2):
                            for n in range(j-1,j+2):
                                scopy=copy.deepcopy(s)
                                child = calmove((i,j),(m,n),scopy)
                                if child == None:continue
                                tmp = AlphaBeta(child,d-1,a,b,False,player)
                                a = max(a,tmp)
                                if a>=b:break
                    else: # khong di chuyen cheo duoc
                        for des in [(i-1,j),(i,j-1),(i,j+1),(i+1,j)]:
                            scopy=copy.deepcopy(s)
                            child = calmove((i,j),des,scopy)
                            if child == None:continue
                            tmp = AlphaBeta(child,d-1,a,b,False,player)
                            a = max(a,tmp)
                            if a>=b:break
        return a
    else:
        for i in range(5):
            for j in range(5):
                if s[i][j]==-player:
                    if (i+j)%2==0: # di chuyen cheo duoc
                        for m in range(i-1,i+2):
                            for n in range(j-1,j+2):
                                scopy=copy.deepcopy(s)
                                child = calmove((i,j),(m,n),scopy)
                                if child == None:continue
                                tmp = AlphaBeta(child,d-1,a,b,True,player)
                                b = min(b,tmp)
                                if a>=b:break
                    else: # khong di chuyen cheo duoc
                        for des in [(i-1,j),(i,j-1),(i,j+1),(i+1,j)]:
                            scopy=copy.deepcopy(s)
                            child = calmove((i,j),des,scopy)
                            if child == None:continue
                            tmp = AlphaBeta(child,d-1,a,b,True,player)
                            b = min(b,tmp)
                            if a>=b:break
        return b

In [2]:
# print("result:",move([[0, -1, 1, 1, 1], [-1, 1, -1, 0, 1], [0, 1, 0, -1, 0], [-1, 1, -1, 0, 0], [0, -1, 0, -1, -1]],1))
# print("result:",move([[0, 1, 1, 1, 1], [1, 1, 0, 0, 1], [-1, 0, 1, 0, -1], [0, 0, 0, 1, 0], [-1, -1, -1, -1, -1]],1))
# print("result:",move([[0, -1, 1, 0, 1], [-1, 0, 0, 1, 1], [-1, -1, 0, 1, 0], [-1, 0, -1, 1, -1], [1, 1, 1, 0, 0]],1))
# print("result:",move([[1, 0, 1, 1, 1], [1, 1, 0, 0, 1], [0, 0, 0, 0, -1], [1, 1, 1, 0, -1], [-1, 0, -1, -1, -1]],1))
print("result:",move([[0, 1, 1, 0, -1], [0, 1, 0, 1, -1], [1, 1, 0, 0, 1], [1, 1, 1, 1, -1], [0, 0, -1, 0, -1]],1))

result: ((0, 2), (0, 3))
